In [1]:
import matplotlib.pyplot as plt
from utils.util_metrics import *
from utils.util_draw import *
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import json
import glob
import natsort as ns
import seaborn as sns
print(plt.style.available)
# import utils.ai_plotting as aiplt
# aiplt.aingura_plot_style(grid_color='white', plt_style='seaborn-v0_8-white')


['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


#### Read Results

In [ ]:
exp = f'results/exp_real/Mfix/tryreals_no_pa/'
# exp = f'results/exp_real/Mfix/tryreals_1pa/'
paths = ns.natsorted(glob.glob(f'{exp}/simu*.json'))
print(paths)
# Load results from JSON files

all_exps = []
all_exps_std = []
models = ['1.SPBN','2.B-SPBN-Simple', '4.B-SPBN-FKDE-Simple', '5.GBN-BIC', '6.GBN-BGe']
# models = ['1.SPBN','2.B-SPBN-Simple', '5.GBN-BIC', '6.GBN-BGe']
for path in paths:
    M = int(path.split('.')[0].split('_')[-1])
    with open(path, 'r') as json_file:
        results = json.load(json_file)
    
        dataframes = []
        model_index, instances_col,simucol = [],[],[]
        for nm, model in enumerate(models):
            instances = ns.natsorted(results[model].keys())
            for n in instances:
                simus = results[model][n].keys()
                for simu in simus:
                    ## PREPARE DATAFRAMES
                    reader = ExperimentsReader(results[model][n][simu], model, 'REF')
                    df_all = reader.return_dataframe(simulated=True).reset_index().drop(columns=['model'], axis=1)
                    
                    reader0 = ExperimentsReader(results[models[0]][n][simu], models[0], 'REF')
                    df0 = reader0.return_dataframe(simulated=True).reset_index()

                    df_all['HC Ratio'] = df0['train']/df_all['train']
                    df_all['Test Ratio'] = df0['test']/df_all['test']
                    df_all['Logl diff'] = df_all['slogl_REF'] - df_all['slogl']
                    

                    df_mean = df_all.mean().to_frame().T
                    df_std = df_all.std().to_frame().T
                    dataframes.append((df_all, df_mean, df_std))
            

                    model_index.append(model[2:])                 
                    instances_col.append(n) 
                    simucol.append(int(simu[-1]))

        all_df= pd.concat([df_mean for _, df_mean,_ in dataframes]).reset_index(drop=True).drop(columns=['index'], axis=1)
        all_df.index = model_index
        all_df.index.name = 'Model'
        all_df['Instances'] = instances_col
        all_df['Dataset'] = simucol
        all_df['M'] = [str(M)]*len(all_df)


        all_df_std= pd.concat([df_std for _, _,df_std in dataframes]).reset_index(drop=True).drop(columns=['index'], axis=1)
        all_df_std.index = model_index
        all_df_std.index.name = 'Model'
        all_df_std['Instances'] = instances_col
        all_df_std['Dataset'] = simucol
        all_df_std['M'] = [str(M)]*len(all_df_std)


        all_exps.append(all_df)
        all_exps_std.append(all_df_std)
        
concat_all = pd.concat(all_exps).reset_index()
concat_all_std = pd.concat(all_exps_std).reset_index()

['results/exp_real/Mfix/tryreals_no_pa copy/simu_all_50.json', 'results/exp_real/Mfix/tryreals_no_pa copy/simu_all_100.json']


#### Prepare Table and CDD

In [27]:
results = concat_all[['Dataset','Model', 'M', 'Instances', 'slogl', 'HC Ratio', 'Test Ratio']].sort_values(by=['Dataset','Model']).reset_index(drop=True).round(2)
results_std = concat_all_std[['Dataset','Model', 'M','Instances', 'slogl', 'HC Ratio', 'Test Ratio']].sort_values(by=['Dataset','Model']).reset_index(drop=True).round(2)

results_with_std = results.copy()
results_with_std['HC Ratio'] =  results['HC Ratio'].astype(str) + ' $\pm$ ' + results_std['HC Ratio'].astype(str)
results_with_std['Test Ratio'] =  results['Test Ratio'].apply(lambda x: f"{x:.2e}").astype(str) + ' $\pm$ ' + results_std['Test Ratio'].astype(str) 
results_with_std['Dataset'] = results_with_std['Dataset'] + 1

##### Ratio Table

In [28]:
results_with_std_bsbns = results_with_std[results_with_std['Model'].str.contains('B-SPBN')].reset_index(drop=True)
# results_with_std_bsbns[['Dataset','Model', 'M', 'HC Ratio', 'Test Ratio']].to_csv(f'{exp}/results_NoPA.csv', index=False)
results_with_std_bsbns

,Dataset,Model,M,Instances,slogl,HC Ratio,Test Ratio
0,1,B-SPBN-FKDE-Simple,50,16384,-1.00,-1981.56 $\pm$ 410.86,-7.70e+00 $\pm$ 0.9
1,1,B-SPBN-FKDE-Simple,100,16384,-1.00,-1852.02 $\pm$ 283.73,-6.76e+00 $\pm$ 0.11
2,1,B-SPBN-Simple,50,16384,-63220.87,1.19 $\pm$ 0.28,1.72e+00 $\pm$ 0.16
3,1,B-SPBN-Simple,100,16384,-62130.04,1.1 $\pm$ 0.17,1.40e+00 $\pm$ 0.09
4,2,B-SPBN-FKDE-Simple,50,16384,-1.00,-356.04 $\pm$ 43.01,-3.14e+00 $\pm$ 0.18
5,2,B-SPBN-FKDE-Simple,100,16384,-1.00,-283.75 $\pm$ 9.18,-2.61e+00 $\pm$ 0.08
6,2,B-SPBN-Simple,50,16384,-41767.58,1.92 $\pm$ 0.23,9.65e+00 $\pm$ 1.02
7,2,B-SPBN-Simple,100,16384,-41272.33,1.45 $\pm$ 0.05,3.49e+00 $\pm$ 0.15
8,3,B-SPBN-FKDE-Simple,50,16384,-1.00,-255.57 $\pm$ 14.56,-2.78e+00 $\pm$ 0.31
9,3,B-SPBN-FKDE-Simple,100,16384,-1.00,-193.33 $\pm$ 11.84,-2.71e+00 $\pm$ 0.34


##### CDD Format
The bergamn-hommel analysis is performed in Rstudio. Then the CDD is plotted in python again

In [32]:
prepare_CD = results_with_std[['Model', 'Dataset','M', 'Instances', 'slogl']]
prepare_CD = prepare_CD[prepare_CD["Model"]!="B-SPBN-FKDE-Simple"]
prepare_CD['data_grid_instances'] = prepare_CD['Dataset'].astype(str) + '_' + prepare_CD['M'] + '_' + prepare_CD['Instances']
slogl_cd = prepare_CD[['Model', 'data_grid_instances', 'slogl']]
slogl_cd.columns = ['network', 'dataset_name', 'metric']
transformed_slogl_cd = slogl_cd.pivot(index='dataset_name', columns='network', values='metric')

path = f"critical_diff/real_no_pa_with78"
os.makedirs(path, exist_ok=True)

transformed_slogl_cd.to_csv(path+ "/slogl.csv", index=False)